### Model fitting

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Read and clean dataset

In [2]:
#data2021 = pd.read_csv('2021_RAW_APC_Data.csv')
data2019 = pd.read_csv('2019_RAW_APC_Data.csv')

In [3]:
for col in ['route finish time','route start time','stop arrival time']:
    data2019[col] = pd.to_datetime(data2019[col])

In [4]:
df = data2019

df['Crowded'] = df['passwithin']>74
df['Supercrowded'] = df['passwithin']>134
df['Capacity'] = df['passwithin']>194
df['UnderNeg5'] = df['passwithin']<-5
df['NegAFew'] = df['passwithin'].between(-5,-1)
df['Over250'] = df['passwithin']>250

df['Crowded000'] = df['passwithin']>0
df['Crowded010'] = df['passwithin']>10
df['Crowded020'] = df['passwithin']>20
df['Crowded030'] = df['passwithin']>30
df['Crowded040'] = df['passwithin']>40
df['Crowded050'] = df['passwithin']>50
df['Crowded060'] = df['passwithin']>60
df['Crowded070'] = df['passwithin']>70
df['Crowded080'] = df['passwithin']>80
df['Crowded090'] = df['passwithin']>90
df['Crowded100'] = df['passwithin']>100
df['Crowded110'] = df['passwithin']>110
df['Crowded120'] = df['passwithin']>120
df['Crowded130'] = df['passwithin']>130
df['Crowded140'] = df['passwithin']>140
df['Crowded150'] = df['passwithin']>150
df['Crowded160'] = df['passwithin']>160
df['Crowded170'] = df['passwithin']>170
df['Crowded180'] = df['passwithin']>180
df['Crowded190'] = df['passwithin']>190
df['Crowded200'] = df['passwithin']>200
df['Crowded210'] = df['passwithin']>210
df['Crowded220'] = df['passwithin']>220
df['Crowded230'] = df['passwithin']>230
df['Crowded240'] = df['passwithin']>240

In [10]:
df = data2019

df['TOD'] = df['stop arrival time'].dt.time
df['DOW'] = df['stop arrival time'].dt.dayofweek # 0 is Monday, 6 is Sunday
df['DOW_name'] = df['stop arrival time'].dt.day_name()
df['Date'] = df['stop arrival time'].dt.date
df['Hour'] = df['stop arrival time'].dt.hour
df['Minute'] = df['stop arrival time'].dt.minute
df['Minute_od'] = df['Hour'] * 60 + df['Minute']
df['Month'] = df['stop arrival time'].dt.month
df['Month_name'] = df['stop arrival time'].dt.month_name()

In [12]:
# First, just see how many rows are in each routedone
rtd = data2019.groupby('routedone').count()['railcar ID']

rtd.name = 'count'

overmuch = rtd[rtd>20]

df01 = data2019[~data2019['routedone'].isin(overmuch.index)]

In [13]:
# df02 is df01 but without >210 observations
df02 = df01[~df01['Crowded210']]

Work with df02

Add DOW, TOD, Time of year, and station

Create ML dataset

Need to create test/train sets**

Readout: need precision and recall.

Precision is TP / (TP+FP). Of all times we predicted the train was crowded, how often was the train crowded?

Recall is TP / (TP+FN). Of all times the train was crowded, how often did we predict that the train was crowded?

F1-score is a weighted average of the two.


In [34]:
# doesn't even take into account station
feats01 = ['Minute_od','DOW','Month']

In [35]:
df = df02
feats = feats01

In [38]:
X = df.loc[:,feats]

y = df.loc[:,'Crowded'].astype(int)

X, y = shuffle(X, y, random_state=19)

In [19]:
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=19)

In [49]:
def cv_readout(model, X, y):
    print('Performing 5-fold cross-validation...')
    scores = cross_validate(model, X, y, cv=5,
                            scoring=['precision','recall','f1'])
    
    print(scores)
    #print('The mean of the RMSEs is', -np.mean(scores['test_score']))
    #print('The standard deviation of the RMSEs is', np.std(scores['test_score'])
    
          # could redo this in the future so that all models can be in one df
          # for now, this df contains just one model's results
    #cv_results = pd.DataFrame(columns=['Precision','Recall','F1'], data= [)         
    #return cv_results
    print(
    np.mean(scores['test_precision']),
    np.mean(scores['test_recall']),
    np.mean(scores['test_f1']),
    np.std(scores['test_precision']),
    np.std(scores['test_recall']),
    np.std(scores['test_f1'])
    )
    
         

## Can I compare with Google's data? statistically?

In [44]:
print(sum(y))

print(len(y))

126670
1295022


## Hello

## Notes 7/15

Can I compare with Google's data statistically?

Note: this dataset is too large. Don't try to fit a Random Forest on the entire dataset (at least this is true on my old laptop!)

Make a scatterplot/pairplot - Day of week and time of day and month - see whether it looks seperable (can we separate Crowded from Non-Crowded based on these variables)

Class imbalance

Cluster in a compelling way

Send PB my visualizations


What makes a train arrival a train arrival

PB: The basic unit is the hour-span at a station. (If there are 3 arrivals during that span, then we average the crowdedness of the 3 arrivals. If there is 1 arrival during that span, then we use the crowdedness of that 1.
-- do 24 hour-spans per day, 365 days in the year = 8760 hour-spans. Then we have 16 stations so our data is 8760 x 16. So essentially we're using PCA to simplify 8760 dimensions down to much fewer. Cool. So we represent the *stations* in the PC-space. Clustering here will tell us if there are essentially 2 types of stations or essentially 3 types of stations, etc. We now better understand types of stations.
-- alternatively, we could seek to understand what types of days there are. We represent each of the 365 days in the year by 24 features per station. Becuase 24 X 16 stations = 384, we're using PCA to simplify 384 dimensions down to much fewer. We represent *days* in the PC-space, and clustering tells us essentially what kinds of days exist.

(In both of the above, the PCs are shorthand for different crowding patterns)

AP: The basic unit is the train arrival. We want to understand what types of train arrivals there are. So, we cluster in the PC-space. But how did we get these Principal components?
-- do some principal components regarding crowding patterns (using nbr boarding, nbr deboarding, nbr within) at the last few stops of this train
---- this should be a different PC-Analysis for each stop
---- clustering then ansswers the question "What kinds of train conditionss are there?
-- Do PCs that capture TOD, DOW, 

Visualize in a way that says what is the appropriate agg strategy.

In [50]:
rf01 = RandomForestClassifier()
scores = cv_readout(rf01,
                    X_train.sample(10000, random_state=19),
                    y_train.sample(10000, random_state=19))
scores

Performing 5-fold cross-validation...
{'fit_time': array([0.43317032, 0.43978477, 0.43702817, 0.43745327, 0.44233346]), 'score_time': array([0.04686999, 0.04863167, 0.04686022, 0.03124833, 0.03125024]), 'test_precision': array([0.25477707, 0.22142857, 0.25503356, 0.25827815, 0.26153846]), 'test_recall': array([0.20512821, 0.15897436, 0.19487179, 0.19897959, 0.17346939]), 'test_f1': array([0.22727273, 0.18507463, 0.22093023, 0.22478386, 0.20858896])}
0.2502111611546143 0.18628466771323915 0.21333008108464457 0.014600852517665082 0.017320872480715992 0.01551900592860361


In [51]:
logi01 = LogisticRegression()
scores = cv_readout(logi01,
                    X_train.sample(10000, random_state=19),
                    y_train.sample(10000, random_state=19))
scores

Performing 5-fold cross-validation...


C:\Users\August\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\August\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\August\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\August\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precis

{'fit_time': array([0.0376544 , 0.04687333, 0.04687047, 0.06065869, 0.05397844]), 'score_time': array([0.01562381, 0.        , 0.01562428, 0.00899768, 0.015625  ]), 'test_precision': array([0., 0., 0., 0., 0.]), 'test_recall': array([0., 0., 0., 0., 0.]), 'test_f1': array([0., 0., 0., 0., 0.])}
0.0 0.0 0.0 0.0 0.0 0.0


C:\Users\August\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [96]:
from sklearn.metrics import SCORERS

sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

*Is there a "soft classification" technique? I.e. something where we want to classify over 74 versus under 74, but we penalize farther away errors more?

** that might be a regression problem with an interesting error function

### Word bank

In [102]:
# supervised machine learning
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
import statsmodels.api as sm
import warnings 
#warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR